In [ ]:
-- Track Metrics — Total Listens, Unique Users, Duration, Average Time per User
SELECT
    s.track_id,
    st.report_date,
    COUNT(*) AS total_listens,
    COUNT(DISTINCT st.user_id) AS unique_users,
    SUM(s.duration_ms) AS total_listening_duration,
    ROUND(SUM(s.duration_ms) / COUNT(DISTINCT st.user_id), 2) AS avg_listening_time_per_user
FROM streams AS st
LEFT JOIN songs AS s 
    ON st.track_id = s.track_id
GROUP BY s.track_id, st.report_date
ORDER BY st.report_date, total_listens DESC
LIMIT 10
;


In [ ]:
--Top 3 songs per genre
SELECT *
FROM (
    SELECT
        s.track_genre,
        st.report_date,
        s.track_id,
        s.artists,
        COUNT(*) AS total_listens,
        DENSE_RANK() OVER (
            PARTITION BY s.track_genre, st.report_date 
            ORDER BY COUNT(*) DESC
        ) AS song_rank
    FROM streams AS st
    LEFT JOIN songs AS s 
        ON st.track_id = s.track_id
    GROUP BY s.track_genre, st.report_date, s.track_id, s.artists
)
WHERE song_rank <= 3
ORDER BY report_date, track_genre, song_rank LIMIT 10;


In [ ]:
--Top 5 overall across all days.
SELECT *
FROM (
    SELECT 
        s.track_genre,
        COUNT(*) AS total_listens_overall,
        SUM(s.duration_ms) AS total_listening_duration_overall,
        COUNT(DISTINCT s.track_id) AS unique_tracks,
        DENSE_RANK() OVER (ORDER BY COUNT(*) DESC) AS rank
    FROM streams AS st
    LEFT JOIN songs AS s 
        ON st.track_id = s.track_id
    WHERE s.track_genre IS NOT NULL
    GROUP BY s.track_genre
) AS ranked_genres
WHERE rank <= 5
ORDER BY rank;
